<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_2/blob/main/Crop_unilateral_eyelid_olympia_202503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Crop_unilateral_eyelid_olympia_202503**

In [2]:
# prompt: gdriveをマウント

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Olympia_new
zip_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Olympia_new_dataset.zip"

# Heltel_list
Heltel_list_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/個人情報なし　コントロールと活動期.xlsx"

# YOLO11l
model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"